In [ ]:
import pylxd
import os

from pylxd import Client

## Authenticating

In [ ]:
BASE_DIR = os.getcwd()
def cert_path(file):
    return os.path.join(BASE_DIR,'backend', 'test_certificate', file)

/Users/pannapat/Desktop/cloudboi/testcertificates/test.crt


## Before executing this need to make sure that
core.trust_password is set to expected password

```
sudo lxc config get core.trust_password
```

```
sudo lxc config set core.trust_password test
```

# or use following command to get trusted token

```
lxc config trust add
```



In [ ]:
cert = (cert_path('lxd.crt'),cert_path('lxd.key'))
endpoint = 'https://198.19.249.45:8443'

trusted = 'eyJjbGllbnRfbmFtZSI6InB5dGhvbnJlcGwiLCJmaW5nZXJwcmludCI6IjM1NmEyYzhhNDJlMjFlMDhjMDI2MzdiNzUzOTBkOTk3NmYzMGU5YzBmNjU3YzYxNWEwMmU0Y2YyMjhlMTYwZDciLCJhZGRyZXNzZXMiOlsiMTk4LjE5LjI0OS40NTo4NDQzIl0sInNlY3JldCI6IjI3ZTQ5ZjNhOGEzZTZlNGQ2YzI2MzBlMGVlNDQ5YzY3NmI3MzI3MzYyMTBlODA3OTA0NzliNTBmOTU3ZTU3NzkiLCJleHBpcmVzX2F0IjoiMDAwMS0wMS0wMVQwMDowMDowMFoifQ=='

client: pylxd.Client = pylxd.Client(endpoint=endpoint, cert=cert, verify=False)

client.authenticate(trusted)
print(client.trusted)

True


In [ ]:

allMembersInCluster = client.cluster.members.all()

for member in allMembersInCluster:
    print(member.server_name)


main
sub-0
sub-1


In [ ]:

def get_instance(instance_name):  
  try:
    instance = client.instances.get(instance_name)
    return instance
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return None
  
test_instance = get_instance('test-instance')

Instance not found


In [ ]:
all_image = client.images.all()

for image in all_image:
    print(image.alias)
    print(image.fingerprint)

In [ ]:
def create_instance(instance_name):
  instance_info = {
      'name': instance_name,
      "source": {
            "type": "image",
            "mode": "pull",
            "server": "https://cloud-images.ubuntu.com/releases",
            "protocol": "simplestreams",
            "alias": "22.04",
        },
      'profiles': ['default'],
      'config': {
        'limits.cpu': '1',
        'limits.memory': '256MB'
      }
    }
  try:
    instance = client.instances.create(instance_info, wait=True)
    return instance
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
  
test_instance = create_instance('test-instance')

print(get_instance('test-instance'))


In [ ]:
def start_instance(instance_name):
  try:
    instance = get_instance(instance_name)
    instance.start(wait=False)
    return instance
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
  
print(start_instance('test-instance'))

In [ ]:
def delete_instance(instance_name):
  try:
    instance = client.instances.get(instance_name)
    instance.delete(wait=True)
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
# Cant delete instance if it is running
delete_instance('test-instance')

Instance is running


In [ ]:
# execute command 
def execute_command(instance_name, command):
  try:
    instance = get_instance(instance_name)
    result = instance.execute(command)
    return result
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
  
print(execute_command('test-instance', ['ls', '-l']))

Traceback (most recent call last):
  File "/Users/pannapat/.vscode/extensions/ms-python.python-2024.20.0-darwin-arm64/python_files/python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 11, in <module>
  File "<string>", line 5, in execute_command
  File "/Users/pannapat/Desktop/cloudboi/backend/.venv/lib/python3.12/site-packages/pylxd/models/instance.py", line 475, in execute
    stdin.connect()
  File "/Users/pannapat/Desktop/cloudboi/backend/.venv/lib/python3.12/site-packages/ws4py/client/__init__.py", line 214, in connect
    self.sock = ssl.wrap_socket(self.sock, **self.ssl_options)
                ^^^^^^^^^^^^^^^
AttributeError: module 'ssl' has no attribute 'wrap_socket'

